In [1]:
import numpy as np
import cv2
import os
import time
from natsort import natsort_keygen
from IPython.display import clear_output
import sys
def nothing(useless=None):
    pass

def run():
    cv2.namedWindow("Mask")
    cap = cv2.VideoCapture(0)

    #Defining the trackbars
    cv2.createTrackbar('R_l','Mask',26,255,nothing)
    cv2.createTrackbar('G_l','Mask',46,255,nothing)
    cv2.createTrackbar('B_l','Mask',68,255,nothing)

    cv2.createTrackbar('R_h','Mask',108,255,nothing)
    cv2.createTrackbar('G_h','Mask',138,255,nothing)
    cv2.createTrackbar('B_h','Mask',155,255,nothing)

    while True:

        #Getting the position of the trackbads
        R_l = cv2.getTrackbarPos('R_l', 'Mask')
        G_l = cv2.getTrackbarPos('G_l', 'Mask')
        B_l = cv2.getTrackbarPos('B_l', 'Mask')

        R_h = cv2.getTrackbarPos('R_h', 'Mask')
        G_h = cv2.getTrackbarPos('G_h', 'Mask')
        B_h = cv2.getTrackbarPos('B_h', 'Mask')

        #Getting frame, blurring it and converting rgb to hsv
        _,frame = cap.read()
        blurred_frame = cv2.blur(frame,(5,5),0)    
        hsv_frame = cv2.cvtColor(blurred_frame,cv2.COLOR_BGR2HSV)

        #Defining color theshold
        low_green = np.array([R_l, G_l, B_l])
        high_green = np.array([R_h, G_h, B_h])
        green_mask = cv2.inRange(hsv_frame, low_green, high_green)

        #Morphological adjestments
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        opening = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=1)

        #Getting the largest contour
        contours,_ = cv2.findContours(green_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)    

        try:

            biggest = sorted(contours,key=cv2.contourArea,reverse=True)[0]
            cv2.drawContours(frame,biggest,-1,(255,0,0),1)

            #Creating blank mask and filling in the contour
            blank_mask = np.zeros(frame.shape, dtype=np.uint8)
            cv2.fillPoly(blank_mask, [biggest], (255,255,255))
            blank_mask = cv2.cvtColor(blank_mask, cv2.COLOR_BGR2GRAY)
            result = cv2.bitwise_and(frame,frame,mask=blank_mask)

            x,y,w,h = cv2.boundingRect(blank_mask)
            ROI = result[y:y+h, x:x+w]

            cv2.imshow('Mask', ROI)
            cv2.imshow('frame', frame)

        except IndexError:
            continue

        key = cv2.waitKey(1)
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    
# default = {'R_l': 26, 'G_l': 46, 'B_l': 68, 'R_h': 108, 'G_h': 138, 'B_h': 155}
# default = {'R_l': 31, 'G_l': 26, 'B_l': 54, 'R_h': 110, 'G_h': 139, 'B_h': 145}
default = {'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}

def foo(folder_path,path,save_path,dic = default):
    
    R_l = dic['R_l']
    G_l = dic['G_l']
    B_l = dic['B_l'] 

    R_h = dic['R_h']
    G_h = dic['G_h']
    B_h = dic['B_h']
    
    while True:

        #Getting frame, blurring it and converting rgb to hsv
        frame =  cv2.imread(folder_path+'/'+path)    
        blurred_frame = cv2.blur(frame,(5,5),0)    
        hsv_frame = cv2.cvtColor(blurred_frame,cv2.COLOR_BGR2HSV)

        #Defining color theshold
        low_green = np.array([R_l, G_l, B_l])
        high_green = np.array([R_h, G_h, B_h])
        green_mask = cv2.inRange(hsv_frame, low_green, high_green)

        #Morphological adjestments
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        opening = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=1)

        #Getting the largest contour
        contours,_ = cv2.findContours(green_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)    

        try:
            biggest = sorted(contours,key=cv2.contourArea,reverse=True)[0]
            cv2.drawContours(frame,biggest,-1,(0,0,0),1)

            #Creating blank mask and filling in the contour
            blank_mask = np.zeros(frame.shape, dtype=np.uint8)
            cv2.fillPoly(blank_mask, [biggest], (255,255,255))
            blank_mask = cv2.cvtColor(blank_mask, cv2.COLOR_BGR2GRAY)
            result = cv2.bitwise_and(frame,frame,mask=blank_mask)

            x,y,w,h = cv2.boundingRect(blank_mask)
            ROI = result[y:y+h, x:x+w]

        except IndexError:
            continue
            
        image_name = '{0}/{1}'.format(save_path,path)
        cv2.imwrite(image_name,ROI)
        break

def foo2(folder_path,path,dic = default):
    cv2.namedWindow('Mask')    
    #Defining the trackbars
    cv2.createTrackbar('R_l','Mask',dic['R_l'],255,nothing)
    cv2.createTrackbar('G_l','Mask',dic['G_l'],255,nothing)
    cv2.createTrackbar('B_l','Mask',dic['B_l'],255,nothing)

    cv2.createTrackbar('R_h','Mask',dic['R_h'],255,nothing)
    cv2.createTrackbar('G_h','Mask',dic['G_h'],255,nothing)
    cv2.createTrackbar('B_h','Mask',dic['B_h'],255,nothing)

    while True:
        #Getting the position of the trackbads
        R_l = cv2.getTrackbarPos('R_l', 'Mask')
        G_l = cv2.getTrackbarPos('G_l', 'Mask')
        B_l = cv2.getTrackbarPos('B_l', 'Mask')

        R_h = cv2.getTrackbarPos('R_h', 'Mask')
        G_h = cv2.getTrackbarPos('G_h', 'Mask')
        B_h = cv2.getTrackbarPos('B_h', 'Mask')
        
        #Getting frame, blurring it and converting rgb to hsv
        frame =  cv2.imread(folder_path+'/'+path)    
        blurred_frame = cv2.blur(frame,(5,5),0)    
        hsv_frame = cv2.cvtColor(blurred_frame,cv2.COLOR_BGR2HSV)

        #Defining color theshold
        low_green = np.array([R_l, G_l, B_l])
        high_green = np.array([R_h, G_h, B_h])
        green_mask = cv2.inRange(hsv_frame, low_green, high_green)

        #Morphological adjestments
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        opening = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=1)

        #Getting the largest contour
        contours,_ = cv2.findContours(green_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)    

        try:
            biggest = sorted(contours,key=cv2.contourArea,reverse=True)[0]
            cv2.drawContours(frame,biggest,-1,(0,0,0),1)

            #Creating blank mask and filling in the contour
            blank_mask = np.zeros(frame.shape, dtype=np.uint8)
            cv2.fillPoly(blank_mask, [biggest], (255,255,255))
            blank_mask = cv2.cvtColor(blank_mask, cv2.COLOR_BGR2GRAY)
            result = cv2.bitwise_and(frame,frame,mask=blank_mask)

            x,y,w,h = cv2.boundingRect(blank_mask)
            ROI = result[y:y+h, x:x+w]
        
            cv2.imshow('frame', frame)
            cv2.imshow('Mask', ROI)

        except IndexError:
            print('error ran')
            key = cv2.waitKey(1)
            if key == 27:    
                d = {}
                d['R_l'] = R_l
                d['G_l'] = G_l
                d['B_l'] = B_l
                d['R_h'] = R_h
                d['G_h'] = G_h
                d['B_h'] = B_h

                cv2.destroyAllWindows()
                return d
            continue

        key = cv2.waitKey(1)
        if key == 27:    
            d = {}
            d['R_l'] = R_l
            d['G_l'] = G_l
            d['B_l'] = B_l
            d['R_h'] = R_h
            d['G_h'] = G_h
            d['B_h'] = B_h
            
            cv2.destroyAllWindows()
            return d

In [2]:
store = []
store.append({'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255})
perfection={}
perfection['Apple   Apple scab'] = {'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Apple   Black rot'] = {'R_l': 4, 'G_l': 6, 'B_l': 9, 'R_h': 111, 'G_h': 255, 'B_h': 255}
perfection['Apple   healthy'] = {'R_l': 16, 'G_l': 10, 'B_l': 10, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Cherry (including sour)   healthy'] = {'R_l': 17, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Cherry (including sour)   Powdery mildew'] = {'R_l': 33, 'G_l': 12, 'B_l': 20, 'R_h': 114, 'G_h': 255, 'B_h': 231}
perfection['Corn (maize)   Cercospora leaf spot Gray leaf spot'] = {'R_l': 12, 'G_l': 12, 'B_l': 17, 'R_h': 114, 'G_h': 255, 'B_h': 231}
perfection['Corn (maize)   Common rust'] = {'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Corn (maize)   healthy'] = {'R_l': 0, 'G_l': 0, 'B_l': 0, 'R_h': 255, 'G_h': 255, 'B_h': 255}
perfection['Corn (maize)   Northern Leaf Blight'] = {'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Grape   Black rot'] = {'R_l': 14, 'G_l': 20, 'B_l': 40, 'R_h': 94, 'G_h': 255, 'B_h': 255}
perfection['Grape   Esca (Black Measles)'] = {'R_l': 6, 'G_l': 21, 'B_l': 20, 'R_h': 91, 'G_h': 255, 'B_h': 255}
perfection['Grape   healthy']= {'R_l': 34, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Grape   Leaf blight (Isariopsis Leaf Spot)'] = {'R_l': 6, 'G_l': 63, 'B_l': 34, 'R_h': 93, 'G_h': 255, 'B_h': 255}
perfection['Blueberry   healthy'] = {'R_l': 0, 'G_l': 19, 'B_l': 20, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Orange   Haunglongbing (Citrus greening)'] = {'R_l': 14, 'G_l': 28, 'B_l': 34, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Peach   Bacterial spot'] = {'R_l': 0, 'G_l': 21, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Peach   healthy']= {'R_l': 14, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Pepper, bell   Bacterial spot'] = {'R_l': 16, 'G_l': 20, 'B_l': 23, 'R_h': 108, 'G_h': 255, 'B_h': 231}
perfection['Pepper, bell   healthy'] = {'R_l': 23, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Potato   Early blight'] = {'R_l': 0, 'G_l': 15, 'B_l': 39, 'R_h': 100, 'G_h': 255, 'B_h': 255}
# perfection['Potato   healthy'] = {'R_l': 9, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Potato   Late blight'] = {'R_l': 7, 'G_l': 7, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Raspberry   healthy'] = {'R_l': 23, 'G_l': 13, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Soybean   healthy'] = {'R_l': 25, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Squash   Powdery mildew'] = {'R_l': 15, 'G_l': 20, 'B_l': 80, 'R_h': 105, 'G_h': 255, 'B_h': 255}
perfection['Tomato   Bacterial spot'] = {'R_l': 21, 'G_l': 49, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Tomato   Early blight'] = {'R_l': 13, 'G_l': 28, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Tomato   healthy'] = {'R_l': 0, 'G_l': 25, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Tomato   Late blight'] = {'R_l': 13, 'G_l': 48, 'B_l': 0, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Tomato   Leaf Mold'] = {'R_l': 17, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Tomato   Septoria leaf spot'] = {'R_l': 16, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Tomato   Spider mites Two-spotted spider mite'] = {'R_l': 17, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Tomato   Target Spot'] = {'R_l': 15, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Tomato   Tomato mosaic virus'] = {'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Apple   Cedar apple rust'] = {'R_l': 19, 'G_l': 34, 'B_l': 36, 'R_h': 101, 'G_h': 255, 'B_h': 255}
perfection['Strawberry   healthy'] = {'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}
perfection['Tomato   Tomato Yellow Leaf Curl Virus'] = {'R_l': 29, 'G_l': 26, 'B_l': 0, 'R_h': 100, 'G_h': 255, 'B_h': 255}


In [3]:
# with open('perfection.txt','a+') as f:
#             f.write('---------------------------------------')

In [5]:
source_path = 'PlantVillage-Dataset-renamed'
save_path = 'New-Dataset-Segmented'


d = {'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}

def minifoo(files,source_folder_path,d,store,path_to_save,position = 0):
    
    
    last = store[-1]
    d = foo2(source_folder_path,files[int(position)],d)
    store.append(d)
    print(d)
    for file_path in files:
        foo(source_folder_path,file_path,path_to_save,dic=d)
        
    print(folder_path,'done.')
    flag = input('Are you satisfied with the result? ')
    
    
    if flag == 'n':
        last = store.pop()
        minifoo(files,source_folder_path,d,store,path_to_save)
    elif flag.isnumeric():
        last = store.pop()
        minifoo(files,source_folder_path,d,store,path_to_save,position = flag)
    elif flag == 'kill':
        return flag
    elif flag == 'back':
        store.append(last)
        minifoo(files,source_folder_path,last,store,path_to_save)
    elif flag == 'perfect':
        temp = os.path.split(source_folder_path)[-1]
        perfection['{}'.format(temp)] = d
        with open('perfection.txt','a+') as f:
            f.write('{0} : {1}\n'.format(temp,d))
        print('thanks! {0} : {1}'.format(temp,d))
    elif flag == 'god please make it stop':
        return flag

# make the save_path
if not os.path.exists(save_path):
    os.mkdir(save_path)
    
    
for folder_path in os.listdir(source_path):
    
    try:
        print(folder_path)
        temp_ = perfection['{}'.format(folder_path)]
        print(temp_)
#         for file_path in files:
#             foo(source_folder_path,file_path,path_to_save,dic=temp_)
        print(folder_path,'achived perfection!!!')
    except:
        #make subfolders in the save_path
        sub_folder_path = os.path.join(save_path,folder_path)
        if not os.path.exists(sub_folder_path):
            os.mkdir(sub_folder_path)


        source_folder_path = os.path.join(source_path,folder_path)
        files = os.listdir(source_folder_path)
        files.sort(key=natsort_keygen(key=lambda y: y.lower()))
        path_to_save = os.path.join(save_path,folder_path)


        flag = minifoo(files,source_folder_path,d,store,path_to_save)
        if flag == 'kill':
            print('PROCESS KILLED!!!')
            break
        if flag == 'god please make it stop':
            print('God had mercy on your soal...',folder_path,'skipped.')

apple_black_rot_1.jpg


NotADirectoryError: [WinError 267] The directory name is invalid: 'Test images/New folder\\apple_black_rot_1.jpg'

In [20]:
folder_path = 'Apple   Apple scab'
perfection['{}'.format(folder_path)]

{'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}

In [15]:
perfection['Apple   Apple scab']

{'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}

In [ ]:
# {'R_l': 6, 'G_l': 15, 'B_l': 14, 'R_h': 106, 'G_h': 255, 'B_h': 255},

In [108]:
# store = []

In [125]:
position = 0
home_dir = 'New-Dataset/Apple   Apple scab'
d = foo2(home_dir,files[position],d)
store.append(d)
print(d)

killed
{'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}


In [131]:
# [{'R_l': 38, 'G_l': 24, 'B_l': 50, 'R_h': 108, 'G_h': 122, 'B_h': 155},
#  {'R_l': 38, 'G_l': 24, 'B_l': 50, 'R_h': 108, 'G_h': 122, 'B_h': 155},
#  {'R_l': 3, 'G_l': 12, 'B_l': 57, 'R_h': 112, 'G_h': 109, 'B_h': 141},
#  {'R_l': 8, 'G_l': 18, 'B_l': 61, 'R_h': 128, 'G_h': 159, 'B_h': 140},
#  {'R_l': 20, 'G_l': 42, 'B_l': 56, 'R_h': 131, 'G_h': 146, 'B_h': 126},
#  {'R_l': 112, 'G_l': 30, 'B_l': 121, 'R_h': 143, 'G_h': 93, 'B_h': 176},
#  {'R_l': 50, 'G_l': 64, 'B_l': 51, 'R_h': 111, 'G_h': 146, 'B_h': 133},
#  {'R_l': 45, 'G_l': 24, 'B_l': 54, 'R_h': 106, 'G_h': 134, 'B_h': 154},
#  {'R_l': 50, 'G_l': 15, 'B_l': 40, 'R_h': 106, 'G_h': 126, 'B_h': 143},
#  {'R_l': 31, 'G_l': 15, 'B_l': 48, 'R_h': 113, 'G_h': 196, 'B_h': 157},
#  {'R_l': 31, 'G_l': 26, 'B_l': 57, 'R_h': 99, 'G_h': 111, 'B_h': 133},
#  {'R_l': 10, 'G_l': 27, 'B_l': 61, 'R_h': 122, 'G_h': 184, 'B_h': 156},
#  {'R_l': 17, 'G_l': 48, 'B_l': 66, 'R_h': 96, 'G_h': 121, 'B_h': 148},
#  {'R_l': 27, 'G_l': 29, 'B_l': 39, 'R_h': 121, 'G_h': 179, 'B_h': 145},
#  {'R_l': 35, 'G_l': 27, 'B_l': 54, 'R_h': 114, 'G_h': 155, 'B_h': 123},
#  {'R_l': 57, 'G_l': 21, 'B_l': 18, 'R_h': 113, 'G_h': 123, 'B_h': 138},
#  {'R_l': 13, 'G_l': 15, 'B_l': 61, 'R_h': 96, 'G_h': 152, 'B_h': 164},
#  {'R_l': 10, 'G_l': 19, 'B_l': 42, 'R_h': 95, 'G_h': 161, 'B_h': 129},
#  {'R_l': 6, 'G_l': 25, 'B_l': 50, 'R_h': 74, 'G_h': 117, 'B_h': 149},
#  {'R_l': 0, 'G_l': 10, 'B_l': 36, 'R_h': 100, 'G_h': 255, 'B_h': 255}]

In [130]:
store

[{'R_l': 38, 'G_l': 24, 'B_l': 50, 'R_h': 108, 'G_h': 122, 'B_h': 155},
 {'R_l': 38, 'G_l': 24, 'B_l': 50, 'R_h': 108, 'G_h': 122, 'B_h': 155},
 {'R_l': 3, 'G_l': 12, 'B_l': 57, 'R_h': 112, 'G_h': 109, 'B_h': 141},
 {'R_l': 8, 'G_l': 18, 'B_l': 61, 'R_h': 128, 'G_h': 159, 'B_h': 140},
 {'R_l': 20, 'G_l': 42, 'B_l': 56, 'R_h': 131, 'G_h': 146, 'B_h': 126},
 {'R_l': 112, 'G_l': 30, 'B_l': 121, 'R_h': 143, 'G_h': 93, 'B_h': 176},
 {'R_l': 50, 'G_l': 64, 'B_l': 51, 'R_h': 111, 'G_h': 146, 'B_h': 133},
 {'R_l': 45, 'G_l': 24, 'B_l': 54, 'R_h': 106, 'G_h': 134, 'B_h': 154},
 {'R_l': 50, 'G_l': 15, 'B_l': 40, 'R_h': 106, 'G_h': 126, 'B_h': 143},
 {'R_l': 31, 'G_l': 15, 'B_l': 48, 'R_h': 113, 'G_h': 196, 'B_h': 157},
 {'R_l': 31, 'G_l': 26, 'B_l': 57, 'R_h': 99, 'G_h': 111, 'B_h': 133},
 {'R_l': 10, 'G_l': 27, 'B_l': 61, 'R_h': 122, 'G_h': 184, 'B_h': 156},
 {'R_l': 17, 'G_l': 48, 'B_l': 66, 'R_h': 96, 'G_h': 121, 'B_h': 148},
 {'R_l': 27, 'G_l': 29, 'B_l': 39, 'R_h': 121, 'G_h': 179, 'B_h': 1

In [132]:
sum_R_l = 0
sum_G_l = 0
sum_B_l = 0
sum_R_h = 0
sum_G_h = 0
sum_B_h = 0

length = len(store)    
for i in range(length):
    sum_R_l  = sum_R_l + store[i]['R_l']
    sum_G_l  = sum_G_l + store[i]['G_l']
    sum_B_l  = sum_B_l + store[i]['B_l']
    sum_R_h  = sum_R_h + store[i]['R_h']
    sum_G_h  = sum_G_h + store[i]['G_h']
    sum_B_h  = sum_B_h + store[i]['B_h']
default['R_l'] = int(sum_R_l/length)
default['G_l'] = int(sum_G_l/length)
default['B_l'] = int(sum_B_l/length)
default['R_h'] = int(sum_R_h/length)
default['G_h'] = int(sum_G_h/length)
default['B_h'] = int(sum_B_h/length)
print(default)

{'R_l': 30, 'G_l': 25, 'B_l': 53, 'R_h': 109, 'G_h': 145, 'B_h': 151}


In [10]:
cv2.destroyAllWindows()

In [11]:
folder_path = 'Test images/New folder'
path = 'apple_cedar_rust_1.jpg'

foo2(folder_path,path,dic = default)

{'R_l': 8, 'G_l': 47, 'B_l': 98, 'R_h': 100, 'G_h': 211, 'B_h': 219}